In [2]:
%reload_ext autoreload
%autoreload 2

import pandas as pd

In [3]:
from pathlib import Path

from model import (
    extract_emp_data,
    extract_trip_data,
    extract_tour_data,
)
from totals import summarize_totals
from rates import summarize_rates

In [4]:
dir_ = Path('T:/data/base_2022/CVM/Calib_CVM_B/')

In [5]:
emp = extract_emp_data(path=dir_/'model_output/Zonal Properties CVM.csv')
emp

,emp
industry,
Industrial,207356
Retail,144185
Service/Gov/Office/FA,1066343
Transport,37037
Wholesale,43668


In [6]:
tours = extract_tour_data(path=dir_/'model_output/commercialVehicleTours.csv')
tours.head(2)

,industry,mode
tour_id,,
1,Industrial,Light Heavy Duty Truck
2,Industrial,Drive Alone


In [7]:
tours.industry.unique()

array(['Industrial', 'Service/Gov/Office/FA', 'Transport', 'Retail',
       'Wholesale'], dtype=object)

In [8]:
tours['mode'].unique()

array(['Light Heavy Duty Truck', 'Drive Alone', 'Heavy Heavy Duty Truck',
       'Medium Heavy Duty Truck'], dtype=object)

In [9]:
trips = extract_trip_data(path=dir_/'model_output/commercialVehicleTrips.csv')

# add industry and tour mode (tour-level data assignment) to trip data
trips = trips.reset_index().merge(
    tours,
    how='left',
    on='tour_id'
).set_index('trip_id')

trips.head(2)

,tour_id,stop_id,weight_trip,weight_person,distance,industry,mode
trip_id,,,,,,,
1,1,1,3.36,3.36,13.258280,Industrial,Light Heavy Duty Truck
2,1,2,3.36,3.36,13.258282,Industrial,Light Heavy Duty Truck


In [22]:
trips['mode'].value_counts()

Light Heavy Duty Truck     626286
Drive Alone                395083
Medium Heavy Duty Truck    201810
Heavy Heavy Duty Truck     106959
Name: mode, dtype: int64

In [26]:
trips.assign(vmt=(trips['distance'] * trips['weight_person'])).groupby('mode').vmt.sum().sort_index()

mode
Drive Alone                3.177168e+05
Heavy Heavy Duty Truck     1.026740e+06
Light Heavy Duty Truck     8.946066e+06
Medium Heavy Duty Truck    6.211732e+05
Name: vmt, dtype: float64

In [29]:
totals = summarize_totals(emp=emp, tours=tours, trips=trips)
totals.to_csv(dir_/'model_results_totals.csv')
totals

,emp,tours,trips,vmt,med_heavy_vmt
industry,,,,,
Industrial,207356,44000,148730,1.281649e+06,1.244466e+06
Retail,144185,30836,135518,1.218760e+06,1.176510e+06
Service/Gov/Office/FA,1066343,190970,953850,7.616048e+06,7.401185e+06
Transport,37037,10375,42550,2.636991e+05,2.570520e+05
Wholesale,43668,8810,49490,5.315402e+05,5.147670e+05


In [30]:
rates = summarize_rates(totals=totals)
rates.to_csv(dir_/'model_results_rates.csv')
rates

,tours_per_emp,trips_per_tour,avg_trip_len,pct_light
industry,,,,
Industrial,0.212195,3.380227,8.617286,0.029012
Retail,0.213864,4.394798,8.993347,0.034667
Service/Gov/Office/FA,0.179089,4.994764,7.984534,0.028212
Transport,0.280125,4.101205,6.197393,0.025207
Wholesale,0.201750,5.617480,10.740355,0.031556
